In [17]:
import torch
import torch.nn as nn
from TabNet import TabTransformer
import pandas as pd

In [18]:
trainDF = pd.read_csv("train.csv") 
testDF = pd.read_csv("test.csv")

In [19]:
trainDF.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [20]:
testDF.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,911150,B,14.53,19.34,94.25,659.7,0.08388,0.07800,0.08817,0.02925,...,16.30,28.39,108.10,830.5,0.10890,0.2649,0.3779,0.09594,0.2471,0.07463
1,911157302,M,21.10,20.52,138.10,1384.0,0.09684,0.11750,0.15720,0.11550,...,25.68,32.07,168.20,2022.0,0.13680,0.3101,0.4399,0.22800,0.2268,0.07425
2,9111596,B,11.87,21.54,76.83,432.0,0.06613,0.10640,0.08777,0.02386,...,12.79,28.18,83.51,507.2,0.09457,0.3399,0.3218,0.08750,0.2305,0.09952
3,9111805,M,19.59,25.00,127.70,1191.0,0.10320,0.09871,0.16550,0.09063,...,21.44,30.96,139.80,1421.0,0.15280,0.1845,0.3977,0.14660,0.2293,0.06091
4,9111843,B,12.00,28.23,76.77,442.5,0.08437,0.06450,0.04055,0.01945,...,13.09,37.88,85.07,523.7,0.12080,0.1856,0.1811,0.07116,0.2447,0.08194


In [27]:
from sklearn.model_selection import train_test_split

x = trainDF.values[:,:-1]
y = trainDF.values[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, stratify = y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [33]:
x.shape


(448, 31)

In [34]:
y.shape

(448,)

In [35]:
import numpy as np 
np.max(x,axis=0)

array([901034302, 'M', 28.11, 39.28, 188.5, 2499.0, 0.1447, 0.3454,
       0.4268, 0.2012, 0.304, 0.09744, 2.873, 4.885, 21.98, 525.6,
       0.03113, 0.1354, 0.396, 0.05279, 0.07895, 0.02984, 33.13, 49.54,
       229.3, 3432.0, 0.2226, 1.058, 1.252, 0.291, 0.6638], dtype=object)

In [ ]:
class CustomDataset (Dataset):
    def __init__(self,x,y): 
        self.x = x
        self.y = y
        
    def __len__(self):
        return self.x.shape[0] 
        
    def __getitem__(self, idx):
        return self.x[idx,:], self.y[idx]

batch_size=64
datasetTraining = CustomDataset(x_train,y_train)
datasetTest = CustomDataset(x_test,y_test)

train_dataloader = DataLoader(datasetTraining, batch_size = batch_size, shuffle=True) 
test_dataloader = DataLoader (datasetTest, batch_size-batch_size, shuffle=True)

In [36]:
model = TabTransformer(
    categories = (5, 14, 5, 14, 5, 14, 5, 14, 5, 14),
    num_continuous = 0,
    dim = 32,
    dim_out = 1,
    depth = 6,
    heads =  8,
    attn_dropout = 0.1,
    ff_dropout =  0.1,
    mlp_hidden_mults = (32, 8),
    mlp_act = nn.Softmax(),
)

AttributeError: module 'torch.nn' has no attribute 'Layerform'

In [ ]:
import torch.optim as optim

criterion = nn. CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)

In [ ]:
for epoch in range(50): 
    running_loss= 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data
        labels = labels.reshape(-1, 1)

        optimizer.zero_grad()

        outputs = model(inputs, None)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 2000 === 1999:
            print(f'[{epoch + 1}, {1 + 1:5d}] loss: {running_loss / 2000:.3f}') 
            running_loss = 0.0

print('Finished Training')

In [ ]:
from sklearn.metrics import classification_report

model.eval()
y_pred=[]
y_true=[]


for i, data in enumerate(test_dataloader, 0):
    inputs, labels = data
    outputs = model(inputs, None)
    y_pred.extend(outputs.argmax (dim=1).numpy())
    y_true.extend(labels)

print (classification_report(y_true, y_pred))